## SARSA Algorithm to recommend songs without state and action approximation

In this notebook, we generated the used the autoencoder for state and action space to generate the latent vector representation of both of these entities. Then we used the latent representaion itself as state and action space for learing.

#### Importing necessary libraries for this notebook

In [1]:
import glob
import pandas as pd
import numpy as np
from tqdm import tqdm

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Input, Dense
from keras.models import Model
from keras.models import load_model
import matplotlib.pyplot as plt

from joblib import dump
import random
import math
from numpy import zeros, newaxis
from statistics import mean 
%matplotlib inline

Extracting the latent representation of each track stored in latent_tack_features.csv

In [2]:
latent_track_features_df = pd.read_csv("D:\Recommendation RL\Code\Data\latent_track_features.csv");
latent_track_features_df.head()

,track_id,0,1,2,3,4,5,6,7
0,t_3c2c29b5-c738-42c0-83e1-ee0e88ac834d,0.143063,0.024294,0.004624,-0.091005,0.067549,-0.291867,-0.285134,-0.150432
1,t_f48322ca-044e-4069-a797-4fedbbbbb522,-0.053658,-0.043300,0.071140,-0.061014,-0.044522,-0.037636,0.181544,0.003603
2,t_5487f622-5641-43da-b263-d8954845d163,0.030984,0.090996,-0.081795,-0.066444,0.240973,-0.241382,-0.244575,-0.090051
3,t_5f395f25-5236-4cbd-a41d-74b65e253df7,0.219336,0.094216,0.024554,-0.129817,0.134058,-0.223478,-0.103355,0.034922
4,t_081b193b-03bd-4c30-9255-b127ed5d2ad5,-0.068865,-0.148980,0.055289,-0.048050,-0.025559,-0.199426,-0.077340,-0.000362


Creating a dictionary of track_id and latent vector for easy search

In [3]:
latent_track_features_dict = latent_track_features_df.set_index('track_id').T.to_dict('list')
latent_track_features_dict

{'t_3c2c29b5-c738-42c0-83e1-ee0e88ac834d': [0.1430633068084716,
  0.0242939107120037,
  0.0046235099434852,
  -0.091005265712738,
  0.06754881888628,
  -0.2918666303157806,
  -0.2851338386535644,
  -0.1504323929548263],
 't_f48322ca-044e-4069-a797-4fedbbbbb522': [-0.0536584295332431,
  -0.0432996116578578,
  0.0711399540305137,
  -0.0610139109194278,
  -0.0445215068757534,
  -0.0376364514231681,
  0.1815439611673355,
  0.0036031743511557],
 't_5487f622-5641-43da-b263-d8954845d163': [0.0309836715459823,
  0.0909963399171829,
  -0.0817949175834655,
  -0.0664440244436264,
  0.2409726679325103,
  -0.2413816750049591,
  -0.24457548558712,
  -0.0900507196784019],
 't_5f395f25-5236-4cbd-a41d-74b65e253df7': [0.2193355560302734,
  0.094215989112854,
  0.024554232135415,
  -0.1298169195652008,
  0.134057730436325,
  -0.2234781831502914,
  -0.1033554449677467,
  0.03492221981287],
 't_081b193b-03bd-4c30-9255-b127ed5d2ad5': [-0.0688653588294982,
  -0.1489803791046142,
  0.055288977921009,
  -0.048

Reading session data from session_track_reward_data.csv created during sampling process.

In [4]:
session_df = pd.read_csv("D:\Recommendation RL\Code\Data\session_track_reward_data.csv");
session_df.head()

,session_id,0,1,2,3,4,5,6,7,8,...,30,31,32,33,34,35,36,37,38,39
0,0_00602661-ec08-47d0-b23b-dcd249f45ecc,t_30cb664a-f513-4c4b-918f-5b383a261e12,0,t_e11adb6d-a49f-4614-bf2c-533705bcc29e,0,t_eeac9c96-f4f7-4618-a4b8-092250d6402e,1,t_5c5df2c3-8ce3-42ed-817f-440791f76368,0,t_b3fdb711-268c-472e-a856-3ba616ed0475,...,t_95f71f29-79bb-413b-91d5-78582cb0774c,0,t_b1bedaf2-a919-4f67-b340-15e383ed14e6,0,t_f6775101-da76-43d3-b54e-4a1704c5bfef,0,t_ac1f897f-25c7-4e0e-91b5-1030649dda4b,0,t_f6775101-da76-43d3-b54e-4a1704c5bfef,1
1,0_00646e66-c627-4e8c-b4a5-f68a30b623a0,t_b905c13c-670f-4ae1-a139-84b981045794,0,t_ffc37789-d3ce-4aa6-8e89-067ab0f18b21,0,t_b193cd79-b6cb-4514-8c5b-ead28ac8dd30,0,t_575964e1-d488-4929-9efe-e63ee8586d12,0,t_0ef6d2c8-faf9-4ebe-ac75-2bade8696a3d,...,t_9aef055a-151c-466d-a082-d2826d80ad5c,0,t_71bee55a-863c-41c8-864f-171b40c663d6,0,t_ee116f41-06dc-4f2a-87dc-494235de540e,0,t_244d8a7e-a290-424d-9b2e-05b06426914d,0,t_867d4494-ed16-4d5e-8415-3e2dc60827fc,0
2,0_006d0464-5910-4c4e-b8bf-98db3cd80a9a,t_a2c86617-1149-4a9c-bb0d-d304cd3ef077,0,t_58b65d28-3a54-467e-bf44-511aa89f44d4,0,t_d777aa56-bd42-4df5-ab7b-403526ed1862,1,t_4a6b1494-508d-48eb-a002-c1fad94b1bd8,1,t_77b02acb-1b1f-4b36-b8fc-2c3e01892b9a,...,t_24a69429-5fd7-41aa-a72d-ece8bd618b45,0,t_416f532e-1cb9-4271-a21c-bb5c78e492e9,1,t_7874301a-f2ba-4cc3-a38b-6efddaa89a2d,0,t_00c3f668-2283-447c-94c0-965c5c409f84,0,t_00c3f668-2283-447c-94c0-965c5c409f84,0
3,0_0096cd3a-dc9f-4a6e-aa50-93ed24e17be5,t_ea20035c-6a01-4e09-8c18-2d1e5fff9a61,1,t_02111b05-f8fd-477c-abb4-4bf2e2ed29dc,0,t_ea20035c-6a01-4e09-8c18-2d1e5fff9a61,0,t_279e04f8-5a85-4a83-9831-16d673673456,0,t_ea20035c-6a01-4e09-8c18-2d1e5fff9a61,...,t_0c09bf72-5cac-44d7-9050-dc4ae4ca3a04,0,t_900f7a9e-e082-4378-8705-ecce236cddf6,0,t_0c09bf72-5cac-44d7-9050-dc4ae4ca3a04,1,t_900f7a9e-e082-4378-8705-ecce236cddf6,0,t_0f554e14-572e-4538-a3e9-65c5fee7e580,0
4,0_0101fd8a-ff18-4435-a6f2-c2d2ce608aef,t_938cfd98-d6cf-48cc-a14a-f1426addbf34,0,t_4755ddd1-f2a1-4dfa-bbd4-59dc15de6bbe,0,t_4755ddd1-f2a1-4dfa-bbd4-59dc15de6bbe,1,t_4755ddd1-f2a1-4dfa-bbd4-59dc15de6bbe,1,t_4755ddd1-f2a1-4dfa-bbd4-59dc15de6bbe,...,t_5ee34da6-b6d3-4487-a03e-55b46ec25633,0,t_abcaac58-2a68-4e29-9da1-d50bed637bce,0,t_68897b2f-13fa-43ce-a1d8-1ffb6437a07e,0,t_8f9f2397-334b-48ee-9120-96d9ddf32f92,1,t_4403dd71-7265-4981-a1b9-fca58fa2679b,1


In [5]:
# Loading the autoencode model created to encode state space.

session_dict = session_df.set_index('session_id').T.to_dict('list')

state_encoder = load_model(r'D:\Recommendation RL\Code\Data\pickle\session_state_encoder.h5')

Follwoing function creates a state given list of songs and action performed on that song. To generate ths state stirng, it appends the track_id and action performed with "-" and encoded state is generated using the trained autoencoder.

In [6]:
def construct_state(track_and_reward_list):
    state_matrix = []
    for i in range(5):
        latent_track = latent_track_features_dict[track_and_reward_list[2*i]].copy()
        latent_track.append(track_and_reward_list[2*i + 1])
        state_matrix.append(latent_track)
        
    np_state_matrix = np.array(state_matrix)
    np_state_matrix = np_state_matrix[newaxis, :, :]
    encoded_state = state_encoder.predict(np_state_matrix)
    strings = [str(data) for data in track_and_reward_list]
    state_string = "-".join(strings)
    
    return state_string, encoded_state[0]

Following function evaluates the distance between 2 points in an n-dimensional space. This function is used to find the closest point for state and action approximation.

In [7]:
def get_distance(a, b):
    distance = 0.0
    for i in range(len(a)):
        distance += (a[i] - b[i])**2
    return math.sqrt(distance)

### Policy - 
Policy will lookup at the Q table for a state and will return the action with max Q value. If the Q table is empty, it will return a random action.

If Q table is not empty but state is not present in Q table, the policy function will find the state closest to the current state and return the action based on that state. In this way, we are able to approximate a state using nearest neighbour.

In [8]:
def policy(Q, current_state, action_list, state_vector_dict):
    action_values = Q.get(current_state, dict())
    if len(action_values) != 0:
        return max(action_values, key = action_values.get)
    if len(Q) != 0:
        closest_state = None
        minimum_distance = float('-inf')
        current_state_vector = state_vector_dict[current_state]
        for state in Q:
            dist = get_distance(state_vector_dict[state], current_state_vector)
            if dist < minimum_distance:
                minimum_distance = dist
                closest_state = state
        closest_state_action_values = Q.get(closest_state, dict())
        if len(closest_state_action_values) != 0:
            return max(closest_state_action_values, key = closest_state_action_values.get)
    return random.choice(action_list)

### SARSA Algorithm for Reinforcement Learning

#### Note: This approach is not optimized and therefore will take months to train. Therefore, we explored in other part of this problem some state approximation method.

In [10]:
Q = dict()
state_vector_dict = dict()
action_list = list(latent_track_features_dict.keys())
alpha = 0.5
gamma = 1.0
episode_count = 0
session_action_error_list = []
for session_id in session_dict:
    track_and_reward_list = session_dict[session_id]
    error_list = []
    for i in range(14):
        current_state, encoded_current_state = construct_state(track_and_reward_list[2*i:2*i + 10])
        state_vector_dict[current_state] = encoded_current_state
        current_action_track = track_and_reward_list[2*i + 10]
        reward = track_and_reward_list[2*i + 11]
        
        recommended_action_track = policy(Q, current_state, action_list, state_vector_dict)
        error_list.append(get_distance(latent_track_features_dict[current_action_track],
                                       latent_track_features_dict[recommended_action_track]))
        
        next_state, encoded_next_state = construct_state(track_and_reward_list[2*i + 2 : 2*i + 10 + 2])
        state_vector_dict[next_state] = encoded_next_state
        next_action_track = track_and_reward_list[2*i + 10 + 2]
        
        current_state_action_values = Q.get(current_state, dict())
        next_state_action_values = Q.get(next_state, dict())
        current_state_action_values[current_action_track] = current_state_action_values.get(current_action_track, 0) + alpha * (reward + gamma * next_state_action_values.get(next_action_track, 0) - current_state_action_values.get(current_action_track, 0))
        
        Q[current_state] = current_state_action_values
    
    print("Episode ", episode_count, " mean error: ", mean(error_list))
    print("Error: ", error_list)
    session_action_error_list.append(error_list)
    dump(session_action_error_list, open('session_action_error_list.pkl', 'wb'))
    episode_count += 1

Episode  0  mean error:  0.7624896661566859
Error:  [0.5480777674200736, 0.8471702493381704, 1.0213557622332334, 0.6902569619676384, 1.0386470020601128, 0.477059480067172, 0.6819327890696335, 0.747443770455974, 0.666545783063112, 0.388767753255077, 1.0139031738974718, 0.5530655949060682, 1.1706851124621769, 0.8299441259976885]
Episode  1  mean error:  0.5943635236545759
Error:  [0.6638211013117777, 0.7426471740919485, 0.7280033769870361, 0.39770278500156203, 0.7056121989917404, 0.7222733098703926, 0.734747626292066, 0.758739562695279, 0.6848612509583036, 0.3044691154386249, 0.5988597851833918, 0.7855167603555829, 0.21168703023744972, 0.2821482537489066]
Episode  2  mean error:  0.6379111627610535
Error:  [0.5539017746716498, 0.6561997145395945, 0.6587427298471082, 0.8161671094345287, 0.6891937682501966, 0.6906087962621904, 0.6627351240803016, 0.7634184237091296, 0.6750145612687505, 0.5925548902269941, 0.6692445993416142, 0.33788543862512854, 0.7500162818724762, 0.4150730665250871]
Epis

Episode  25  mean error:  0.5243961184386123
Error:  [0.5094519499226666, 0.2508954042026501, 0.5426253292451391, 0.6073088766451549, 0.19896184027677788, 0.49029265161085167, 0.740194944555865, 0.7021644054841688, 0.7956568472161734, 0.6266062418470743, 0.16043368691170456, 0.8387625846970401, 0.7019894446984479, 0.17620145082685798]
Episode  26  mean error:  0.40201560276973825
Error:  [0.29471610033080353, 0.7399944313583944, 0.24687348374388954, 0.0, 0.0, 0.28012057706556887, 0.6767900390444307, 0.60203758578392, 0.7166355908027692, 0.3413153898893959, 0.4291941200718158, 0.6265582969505247, 0.38586550356102745, 0.28811732017379527]
Episode  27  mean error:  0.5197011903752388
Error:  [0.46156948532827025, 0.238843765151155, 0.6449846524398811, 0.613225100429984, 0.9221592997798176, 0.5931311989368468, 0.43170851636120383, 0.457309709375522, 0.6474945350603625, 0.6097505362863503, 0.61944804201848, 0.3550542988881816, 0.41388135581730917, 0.26725616937997826]
Episode  28  mean erro

Episode  50  mean error:  0.4613719971644985
Error:  [0.705609658852123, 0.10446260567519311, 0.22806776638875945, 0.5102792018371977, 0.4036563755598953, 0.467840324008781, 0.26803986766714827, 0.6483253785020585, 0.24704002950357515, 0.4373258600123051, 0.903669544227891, 0.7971268841711838, 0.3730000631817556, 0.3647644007151123]
Episode  51  mean error:  0.5407071731267816
Error:  [0.5913783360310046, 0.5633961660760924, 0.6455208649056562, 0.4502564456501507, 0.7503429568922249, 0.7667224812769136, 0.21209794889843417, 0.2464983496078995, 0.6742438525214802, 0.5121194116882006, 0.3332751681187921, 0.4684970313406979, 0.5091474467858693, 0.8464039639815258]
Episode  52  mean error:  0.5507018566968983
Error:  [0.7283601979390013, 0.37381675890360283, 0.690924911291774, 0.40569402938991034, 0.4249981213573878, 0.4721408001801878, 0.6941715519670902, 0.3872418267137736, 0.7242105474187878, 0.4900701600687491, 0.7792606640723606, 1.0084475904707122, 0.23156510623537477, 0.298923727747

Episode  75  mean error:  0.4629000102563843
Error:  [0.7561202082179826, 0.7034327081513325, 0.28535212036111945, 0.2338577046601693, 0.18117928745046472, 0.7080584753745381, 0.3479798236791857, 0.37233894291881814, 0.5570968749733756, 0.4860478533887389, 0.26712851372391233, 0.5449154946786454, 0.42178596112638883, 0.6153061748847085]
Episode  76  mean error:  0.525972191975212
Error:  [0.6306669790883604, 0.8286363269844709, 0.698883819317682, 0.697852726560239, 0.6374053451770535, 0.8077990344163133, 0.24354602653276575, 0.18294370525403536, 0.505587148592752, 0.6809191244053663, 0.24649514618574495, 0.3164448036633905, 0.41849572461267515, 0.46793477686211765]
Episode  77  mean error:  0.6285139596992418
Error:  [0.6591128266876858, 0.8641244417163156, 0.8520321911341755, 1.1543291110928229, 0.175118849585805, 0.6246517654332044, 0.2991157711007843, 0.7789971326200447, 0.7344766477204938, 0.23006608911629822, 0.34100291373818736, 0.6402671015504416, 0.7342990561754023, 0.711601538

Episode  100  mean error:  0.5384403290790548
Error:  [0.26051674825747156, 0.9503550717660406, 0.6924159433530662, 0.7746050896632574, 0.4794716604534851, 0.49491310541778966, 0.6607202663226046, 0.7510177109797094, 0.35156707518192315, 0.5187614225994076, 0.15088364734287704, 0.3373775143200346, 0.6268106624895806, 0.4887486889595192]
Episode  101  mean error:  0.481089175544554
Error:  [0.6303130660690861, 0.6000812898427875, 0.4807039576575387, 0.2617240395459336, 0.6207839080851206, 0.3730979922013477, 0.4062390813224097, 0.10308834208692891, 0.8136152818212035, 0.751231482387678, 0.19187463764195753, 0.44754516267972344, 0.547695450329168, 0.5072547659528723]
Episode  102  mean error:  0.5293736401902531
Error:  [0.6749274398640674, 0.1243274651730871, 0.5120749285192123, 0.514908533370031, 0.4833306783277142, 0.4422362258343409, 0.46206923663430266, 0.7275702917106092, 0.492701750902857, 0.6187463045371532, 0.6249500230946209, 0.6822305278253015, 0.3771573164693476, 0.6740002404

KeyboardInterrupt: 